In [4]:
from tkinter import *
import tkinter as tk

root = Tk()


def key(event):
    print("pressed", repr(event.char))


def callback(event):
    print("clicked at", event.x, event.y)

canvas = tk.Canvas(root, width=200, height=200)
frame = Frame(root, width=200, height=200)
frame.focus_set()
frame.bind("<Key>", key)
frame.bind("<Button-1>", callback)
frame.pack()

root.mainloop()

In [5]:
def main():
    root = Tk()
    w = Canvas(root, width=200, height=200, background='white')
    w.pack()
    global y
    y = []

    def _paint(event):
        # event.x 鼠标左键的横坐标
        global y
        # event.y 鼠标左键的纵坐标
        x = []

        x1, y1 = (event.x - 1), (event.y - 1)

        x2, y2 = (event.x + 1), (event.y + 1)

        w.create_oval(x1, y1, x2, y2, fill='red')

        # 鼠标左键一点，就画出了一个小的椭圆

        # 画布与鼠标左键进行绑定
        y.append(y1)

    w.bind("<Button-1>",_paint)

    mainloop()



if __name__ == '__main__':
    main()

In [18]:
import cv2
import numpy as np
img0 = cv2.imread('01.png')
area_left = 720
area_right = 1250
area_up = 300
area_bottom = 800
# avg G&B
aB = 105.9
aG = 157.4
img1 = img0
img =  img1[area_up:area_bottom, area_left:area_right,:]
# img1 =  cv2.resize(img1, (600,500),
                            #    interpolation=cv2.INTER_CUBIC)
# print(img1.shape)
# img1 = gamma(img1,1.5)
img= cv2.medianBlur(img,7)
img= cv2.medianBlur(img, 5)
# img= cv2.medianBlur(img, 5)
B,G= img[:,:,0], img[:,:,1]
B = 255-(np.abs(B-aB))
G = 255-(np.abs(G-aG))
B = B/255
G = G/255
B = np.power(B, 10)
G = np.power(G, 10)
B = B*255
G = G*255
B= cv2.blur(B, (25, 25))
G= cv2.blur(G, (25, 25))

Y =( 3*B +  G)/4

Y = np.floor(Y).round().astype(np.uint8)
Y= cv2.blur(Y, (25, 25))
# Y= cv2.blur(Y, (25, 25))

Y = Y/255
Y = np.power(Y,3)
Y = Y*255
Y = np.floor(Y).round().astype(np.uint8)
Y= cv2.blur(Y, (25, 25))
# Y= cv2.blur(Y, (25, 25))



ret,imgthresh = cv2.threshold(Y,0,255,
            cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)     #Otsu阈值处理,转化为二值图
kernel = np.ones((3,3),np.uint8)                       #定义形态变换卷积核
imgopen = cv2.morphologyEx(imgthresh,cv2.MORPH_OPEN,
            kernel,iterations=2)                       #形态变换：开运算
imgbg = cv2.dilate(imgopen,kernel,iterations=3)        #膨胀操作，确定背景
imgdist = cv2.distanceTransform(imgopen,cv2.DIST_L2,0) #距离转换，用去确定前景
ret,imgfg = cv2.threshold(imgdist,
                    0.7*imgdist.max(),255,2)           #对距离转换结果进行阈值处理
imgfg = np.uint8(imgfg)                                #转换为整数，获得前景
ret,markers = cv2.connectedComponents(imgfg)           #标记阈值处理结果
unknown = cv2.subtract(imgbg,imgfg)                    #确定位置未知区域
markers = markers + 1                                  #加1使背景不为0
markers[unknown == 255] = 0                            #将未知区域设置为0
imgwater = cv2.watershed(img,markers)                  #执行分水岭算法分割图像
img[imgwater == -1] = [0,255,0]     
cv2.imshow('1',img)
cv2.waitKey(0)  # 用于给窗口提供展示时间
cv2.destroyAllWindows()